In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic7/train.csv
/kaggle/input/titanic7/test.csv
/kaggle/input/titanic7/gender_submission.csv


**Exercício 6.1, aproveitamento do código já criado**

1. Importar as bibliotecas que irão ser utilizadas:



In [101]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
np.random.seed(0)

2. Transformar arquivos em dataframe e gerar momentos estatísticos: a. Importar os arquivos “kaggle/input/titanic/test.csv” e “kaggle/input/titanic/train.csv”; b. Gerar momentos estatísticos sobre o dataframe de treino, utilizando “.describe( )”;

In [102]:
train_data = pd.read_csv("/kaggle/input/titanic7/train.csv")
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [103]:
test_data = pd.read_csv("/kaggle/input/titanic7/test.csv")
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


**Exercício 7**

**1. Análise inicial:**
 
 **a. Quais as colunas do banco de dados de teste e de treino?**
 
 **b. Qual o tipo de dado de cada coluna nos dataframes de teste e de treino?**
 
 **c. Qual a quantidade de valores nulos (NaN) em cada feature?**
 
 **d. Realizar um cópia do banco de dados de teste e de treino para que se possa
 fazer a manipulação sem perder informações.**

**Letra a:**

In [104]:
list(train_data.columns)

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [105]:
list(test_data.columns)

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

**letra b:**

In [106]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [107]:
test_data.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

**letra c:**

In [108]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [109]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

**letra d:**

In [110]:
train_data_copia = train_data.copy()

In [111]:
test_data_copia = test_data.copy()

**2. Para lidar com valores nulos, podemos preencher estes valores de alguma forma ou
 descartar a informação. Neste item utilizaremos algumas estratégias para tal.**
 
 **a. A feature “Cabin” contém muitos valores nulos e, assim como “Ticket”, não
 fornece informações relevantes ao modelo à primeira vista. Então pode-se
 retirá-las;**
 
 **b. A coluna “Age” também contém muitos valores nulos, mas dessa vez iremos
 preenchê-los. Utilizar a mediana dos valores de idade para isso, é uma boa
 estratégia inicial;**
 
 **c. A feature “Fare” contém poucos valores nulos, e pode ser útil para análises
 futuras. Pode-se preenchê-la com a mesma estratégia que “Age”, já que
 ambas são contínuas;**
 
 **d. Em “Embarked” também tem-se pouca ocorrência de valores nulos, e como é
 uma feature categórica, preencher com a moda parece menos impactante na
 construção do modelo**

**letra a:**

In [112]:
train_data_copia.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [113]:
train_data_copia.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [114]:
test_data_copia.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [115]:
test_data_copia.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


**letra b:**

In [116]:
train_data_copia['Age'] = train_data_copia['Age'].fillna(train_data_copia['Age'].median())

In [117]:
train_data_copia.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [118]:
test_data_copia['Age'] = test_data_copia['Age'].fillna(test_data_copia['Age'].median())

In [119]:
test_data_copia.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


**letra c:**

In [120]:
train_data_copia['Fare'] = train_data_copia['Fare'].fillna(train_data_copia['Fare'].median())

In [121]:
train_data_copia.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [122]:
test_data_copia['Fare'] = test_data_copia['Fare'].fillna(test_data_copia['Fare'].median())

In [123]:
test_data_copia.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


**letra d:**

In [124]:
train_data_copia['Embarked'] = train_data_copia['Embarked'].fillna(train_data_copia['Embarked'].mode())

In [125]:
train_data_copia.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [126]:
test_data_copia['Embarked'] = test_data_copia['Embarked'].fillna(test_data_copia['Embarked'].mode())

In [127]:
test_data_copia.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


 **3. Para as Features contínuas será útil a criação de grupos para facilitar a análise. Dois métodos do pandas são úteis para esta tarefa, pd.cut e pd.qcut, também é útil visitar esta referência para uma melhor entendimento destes métodos.**
 
 **a. Criar Feature que separe a Feature “Age” em 5 intervalos de mesma extensão;**
 
 **b. Criar Feature que separe “Fare” em 6 intervalos que contenham o mesmo
 número de dados (Não precisam ter a mesma extensão).**

**letra a:**

**Para o train_data:**

In [128]:
train_data_copia.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


**Organizar os valores em ordem crescente baseado na coluna Age:**

In [129]:
train_data_copia_ordenado = train_data_copia.sort_values(by='Age', ascending=False)

In [130]:
train_data_copia_ordenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,30.0000,S
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,7.7750,S
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,34.6542,C
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,49.5042,C
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,7.7500,Q


In [131]:
train_data_copia_ordenado['Faixa de idade'] = pd.cut(train_data_copia_ordenado['Age'], bins=5)

In [132]:
train_data_copia_ordenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,30.0000,S,"(64.084, 80.0]"
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,7.7750,S,"(64.084, 80.0]"
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,34.6542,C,"(64.084, 80.0]"
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,49.5042,C,"(64.084, 80.0]"
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,7.7500,Q,"(64.084, 80.0]"


In [133]:
train_data_copia_ordenado['Classificação'] = pd.cut(train_data_copia_ordenado['Age'], bins=5, labels=['Muito jovem', 'Jovem', 'Adulto', 'Meia-idade', 'Idoso'])

In [134]:
train_data_copia_ordenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade,Classificação
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,30.0000,S,"(64.084, 80.0]",Idoso
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,7.7750,S,"(64.084, 80.0]",Idoso
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,34.6542,C,"(64.084, 80.0]",Idoso
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,49.5042,C,"(64.084, 80.0]",Idoso
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,7.7500,Q,"(64.084, 80.0]",Idoso


**Para o test_data:**

In [135]:
test_data_copia.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


**Organizar os valores em ordem crescente baseado na coluna Age:**

In [136]:
test_data_copia_ordenado = test_data_copia.sort_values(by='Age', ascending=False)

In [137]:
test_data_copia_ordenado.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
96,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,78.8500,S
81,973,1,"Straus, Mr. Isidor",male,67.0,1,0,221.7792,S
305,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,26.5500,S
179,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,83.1583,C
236,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,75.2500,C


In [138]:
test_data_copia_ordenado['Faixa de idade'] = pd.cut(test_data_copia_ordenado['Age'], bins=5)

In [139]:
test_data_copia_ordenado.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade
96,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,78.8500,S,"(60.834, 76.0]"
81,973,1,"Straus, Mr. Isidor",male,67.0,1,0,221.7792,S,"(60.834, 76.0]"
305,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,26.5500,S,"(60.834, 76.0]"
179,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,83.1583,C,"(60.834, 76.0]"
236,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,75.2500,C,"(60.834, 76.0]"


In [140]:
test_data_copia_ordenado['Classificação'] = pd.cut(test_data_copia_ordenado['Age'], bins=5, labels=['Muito jovem', 'Jovem', 'Adulto', 'Meia-idade', 'Idoso'])

In [141]:
test_data_copia_ordenado.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade,Classificação
96,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,78.8500,S,"(60.834, 76.0]",Idoso
81,973,1,"Straus, Mr. Isidor",male,67.0,1,0,221.7792,S,"(60.834, 76.0]",Idoso
305,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,26.5500,S,"(60.834, 76.0]",Idoso
179,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,83.1583,C,"(60.834, 76.0]",Idoso
236,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,75.2500,C,"(60.834, 76.0]",Idoso


**Letra b:**

**Para o train_data:**

In [142]:
train_data_copia_ordenado['Faixa_Fare'] = pd.cut(train_data_copia_ordenado['Fare'], bins=6)

**Nomeando os intervalos:**

In [143]:
labels = ['Muito Baixo', 'Baixo', 'Médio-Baixo', 'Médio-Alto', 'Alto', 'Muito Alto']
train_data_copia_ordenado['Faixa_Fare'] = pd.cut(train_data_copia_ordenado['Fare'], bins=6, labels=labels)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade,Classificação,Faixa_Fare
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.00,0,0,30.0000,S,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
851,852,0,3,"Svensson, Mr. Johan",male,74.00,0,0,7.7750,S,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
96,97,0,1,"Goldschmidt, Mr. George B",male,71.00,0,0,34.6542,C,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.00,0,0,49.5042,C,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
116,117,0,3,"Connors, Mr. Patrick",male,70.50,0,0,7.7500,Q,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,832,1,2,"Richards, Master. George Sibley",male,0.83,1,1,18.7500,S,"(0.34, 16.336]",Muito jovem,"(-0.512, 85.388]"
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,19.2583,C,"(0.34, 16.336]",Muito jovem,"(-0.512, 85.388]"
644,645,1,3,"Baclini, Miss. Eugenie",female,0.75,2,1,19.2583,C,"(0.34, 16.336]",Muito jovem,"(-0.512, 85.388]"
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,14.5000,S,"(0.34, 16.336]",Muito jovem,"(-0.512, 85.388]"


In [144]:
train_data_copia_ordenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade,Classificação,Faixa_Fare
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,30.0000,S,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,7.7750,S,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,34.6542,C,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,49.5042,C,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,7.7500,Q,"(64.084, 80.0]",Idoso,"(-0.512, 85.388]"


**Para o test_data:**

In [146]:
test_data_copia_ordenado['Faixa_Fare'] = pd.cut(test_data_copia_ordenado['Fare'], bins=6)

**Nomeando os intervalos:**

In [147]:
labels = ['Muito Baixo', 'Baixo', 'Médio-Baixo', 'Médio-Alto', 'Alto', 'Muito Alto']
test_data_copia_ordenado['Faixa_Fare'] = pd.cut(test_data_copia_ordenado['Fare'], bins=6, labels=labels)

In [148]:
test_data_copia_ordenado.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Faixa de idade,Classificação,Faixa_Fare
96,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,78.8500,S,"(60.834, 76.0]",Idoso,Muito Baixo
81,973,1,"Straus, Mr. Isidor",male,67.0,1,0,221.7792,S,"(60.834, 76.0]",Idoso,Médio-Baixo
305,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,26.5500,S,"(60.834, 76.0]",Idoso,Muito Baixo
179,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,83.1583,C,"(60.834, 76.0]",Idoso,Muito Baixo
236,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,75.2500,C,"(60.834, 76.0]",Idoso,Muito Baixo
